In [ ]:
# ===========================================
# House Price Prediction - Complete Premium Project
# ===========================================

# %%
# =========================
# Core Libraries
# =========================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# =========================
# Visualization
# =========================
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# =========================
# Data Preprocessing
# =========================
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import category_encoders as ce

# =========================
# Machine Learning Models
# =========================
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# =========================
# Model Evaluation
# =========================
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# =========================
# Advanced Tools
# =========================
from scipy import stats
from scipy.stats import skew
import joblib
import json
import os

# =========================
# Display Settings
# =========================
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.3f}'.format)

print(" All libraries imported successfully!")


# %%
# =========================
# Load Datasets
# =========================
try:
    # Try different possible file paths
    possible_paths = [
        "data/train.csv",
        "train.csv",
        "../data/train.csv",
        "./train.csv"
    ]
    
    train_loaded = False
    test_loaded = False
    
    for path in possible_paths:
        if not train_loaded and os.path.exists(path):
            train = pd.read_csv(path)
            train_loaded = True
            print(f" Train data loaded from: {path}")
            break
    
    for path in possible_paths:
        if not test_loaded and os.path.exists(path.replace('train', 'test')):
            test_path = path.replace('train', 'test')
            test = pd.read_csv(test_path)
            test_loaded = True
            print(f" Test data loaded from: {test_path}")
            break
    
    if not train_loaded:
        print(" No train.csv found. Creating sample data...")
        raise FileNotFoundError
        
except Exception as e:
    # Create sample data for demonstration
    print(" Creating sample data for demonstration...")
    np.random.seed(42)
    n_samples = 1460
    
    train = pd.DataFrame({
        'Id': range(1, n_samples+1),
        'MSSubClass': np.random.choice([20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 150, 160, 180, 190], n_samples),
        'MSZoning': np.random.choice(['RL', 'RM', 'C (all)', 'FV', 'RH'], n_samples, p=[0.7, 0.2, 0.05, 0.03, 0.02]),
        'LotFrontage': np.random.normal(70, 20, n_samples).clip(20, 200),
        'LotArea': np.random.lognormal(9, 0.5, n_samples).clip(1000, 50000),
        'Street': np.random.choice(['Pave', 'Grvl'], n_samples, p=[0.98, 0.02]),
        'Alley': np.random.choice(['NA', 'Grvl', 'Pave'], n_samples, p=[0.94, 0.03, 0.03]),
        'GrLivArea': np.random.normal(1500, 500, n_samples).clip(500, 3000),
        'TotalBsmtSF': np.random.normal(1000, 400, n_samples).clip(0, 3000),
        '1stFlrSF': np.random.normal(1200, 300, n_samples).clip(500, 2500),
        '2ndFlrSF': np.random.normal(500, 300, n_samples).clip(0, 1500),
        'FullBath': np.random.choice([0, 1, 2, 3], n_samples, p=[0.1, 0.4, 0.45, 0.05]),
        'HalfBath': np.random.choice([0, 1, 2], n_samples, p=[0.7, 0.25, 0.05]),
        'BedroomAbvGr': np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.05, 0.2, 0.5, 0.2, 0.05]),
        'KitchenAbvGr': np.random.choice([1, 2, 3], n_samples, p=[0.8, 0.18, 0.02]),
        'TotRmsAbvGrd': np.random.choice([4, 5, 6, 7, 8, 9, 10], n_samples, p=[0.1, 0.2, 0.3, 0.2, 0.1, 0.05, 0.05]),
        'Fireplaces': np.random.choice([0, 1, 2, 3], n_samples, p=[0.5, 0.35, 0.1, 0.05]),
        'GarageCars': np.random.choice([0, 1, 2, 3, 4], n_samples, p=[0.1, 0.3, 0.4, 0.15, 0.05]),
        'GarageArea': np.random.normal(500, 200, n_samples).clip(0, 1000),
        'WoodDeckSF': np.random.exponential(50, n_samples).clip(0, 500),
        'OpenPorchSF': np.random.exponential(40, n_samples).clip(0, 400),
        'EnclosedPorch': np.random.exponential(30, n_samples).clip(0, 300),
        'PoolArea': np.random.choice([0, 200, 300, 400, 500], n_samples, p=[0.95, 0.02, 0.01, 0.01, 0.01]),
        'YrSold': np.random.choice([2006, 2007, 2008, 2009, 2010], n_samples),
        'YearBuilt': np.random.randint(1900, 2010, n_samples),
        'YearRemodAdd': np.random.randint(1950, 2010, n_samples),
        'OverallQual': np.random.randint(1, 11, n_samples),
        'OverallCond': np.random.randint(1, 11, n_samples),
        'SalePrice': np.random.lognormal(12, 0.3, n_samples).clip(30000, 800000)
    })
    
    # Create test data from train (without target)
    test = train.drop('SalePrice', axis=1).copy()
    test['Id'] = range(n_samples + 1, n_samples * 2 + 1)

# Store target variable
target = 'SalePrice'
if target in train.columns:
    y_train = train[target]
else:
    # Create target for sample data if not exists
    y_train = train['SalePrice'] if 'SalePrice' in train.columns else None

train_ids = train['Id'] if 'Id' in train.columns else None
test_ids = test['Id'] if 'Id' in test.columns else None

# Combine train and test for preprocessing
train_test = pd.concat([train.drop(target, axis=1) if target in train.columns else train, test], axis=0).reset_index(drop=True)

print("\n Data Summary:")
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Combined shape: {train_test.shape}")

print("\n First 5 rows of training data:")
display(train.head())

# %% [markdown]
# ## 4.  **Comprehensive EDA**

# %%
# =========================
# Create EDA Report Function
# =========================
def create_eda_report(df, target_col=None):
    """
    Generate comprehensive EDA report
    """
    print("=" * 80)
    print(" COMPREHENSIVE EDA REPORT")
    print("=" * 80)
    
    # 1. Basic Information
    print("\n BASIC INFORMATION")
    print("-" * 40)
    print(f"Shape: {df.shape}")
    print(f"Memory Usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
    
    # 2. Data Types
    print("\n DATA TYPES")
    print("-" * 40)
    dtypes = df.dtypes.value_counts()
    for dtype, count in dtypes.items():
        print(f"{dtype}: {count}")
    
    # 3. Missing Values Analysis
    print("\n MISSING VALUES ANALYSIS")
    print("-" * 40)
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        missing_percent = (missing / len(df)) * 100
        missing_df = pd.DataFrame({
            'Missing_Count': missing,
            'Missing_Percent': missing_percent
        }).sort_values('Missing_Count', ascending=False)
        display(missing_df.head(20))
    else:
        print(" No missing values!")
    
    # 4. Target Variable Analysis
    if target_col and target_col in df.columns:
        print("\n TARGET VARIABLE ANALYSIS")
        print("-" * 40)
        print(f"Target: {target_col}")
        print(f"Mean: ${df[target_col].mean():,.2f}")
        print(f"Median: ${df[target_col].median():,.2f}")
        print(f"Std: ${df[target_col].std():,.2f}")
        print(f"Min: ${df[target_col].min():,.2f}")
        print(f"Max: ${df[target_col].max():,.2f}")
        print(f"Skewness: {df[target_col].skew():.4f}")
        print(f"Kurtosis: {df[target_col].kurtosis():.4f}")
    
    # 5. Statistical Summary
    print("\n STATISTICAL SUMMARY")
    print("-" * 40)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        display(df[numeric_cols].describe().T.head(15))
    else:
        print("No numeric columns found.")
    
    return missing if len(missing) > 0 else pd.Series([])

# Run EDA Report
missing_values = create_eda_report(train, target)

# %% [markdown]
# ## 5. **Advanced Visualizations**

# %%
# =========================
# Advanced Visualization Suite
# =========================
def create_visualizations(df, target_col=None):
    """
    Create comprehensive visualizations
    """
    print(" CREATING ADVANCED VISUALIZATIONS...")
    
    if target_col is None or target_col not in df.columns:
        print(" Target column not found. Creating basic visualizations only.")
        target_col = None
    
    # Get numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Create figure based on available data
    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=('Target Distribution' if target_col else 'Numeric Distribution',
                       'Log Target Distribution' if target_col else 'Feature Distribution',
                       'Feature vs Target' if target_col and 'GrLivArea' in df.columns else 'Feature Correlation',
                       'Feature Distribution',
                       'Correlation Heatmap',
                       'Missing Values Matrix',
                       'Feature Skewness',
                       'Box Plot - Categorical Feature',
                       'Pair Plot - Top Features'),
        specs=[[{'type': 'histogram'}, {'type': 'histogram'}, {'type': 'scatter'}],
               [{'type': 'histogram'}, {'type': 'heatmap'}, {'type': 'heatmap'}],
               [{'type': 'bar'}, {'type': 'box'}, {'type': 'scatter'}]],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # 1. Target/Numeric Distribution
    if target_col:
        fig.add_trace(go.Histogram(x=df[target_col], name='Target',
                                 marker_color='#1f77b4', nbinsx=50),
                    row=1, col=1)
    elif len(numeric_cols) > 0:
        fig.add_trace(go.Histogram(x=df[numeric_cols[0]], name=numeric_cols[0],
                                 marker_color='#1f77b4', nbinsx=50),
                    row=1, col=1)
    
    # 2. Log Target Distribution or Feature Distribution
    if target_col:
        fig.add_trace(go.Histogram(x=np.log1p(df[target_col]), name='Log Target',
                                 marker_color='#ff7f0e', nbinsx=50),
                    row=1, col=2)
    elif len(numeric_cols) > 1:
        fig.add_trace(go.Histogram(x=df[numeric_cols[1]], name=numeric_cols[1],
                                 marker_color='#ff7f0e', nbinsx=50),
                    row=1, col=2)
    
    # 3. Feature vs Target or Feature Correlation
    if target_col and 'GrLivArea' in df.columns:
        fig.add_trace(go.Scatter(x=df['GrLivArea'], y=df[target_col],
                              mode='markers', name='GrLivArea',
                              marker=dict(color='#2ca02c', size=5, opacity=0.6)),
                    row=1, col=3)
    elif len(numeric_cols) > 2:
        fig.add_trace(go.Scatter(x=df[numeric_cols[1]], y=df[numeric_cols[2]],
                              mode='markers', name=f'{numeric_cols[1]} vs {numeric_cols[2]}',
                              marker=dict(color='#2ca02c', size=5, opacity=0.6)),
                    row=1, col=3)
    
    # 4. Feature Distribution
    if len(numeric_cols) > 3:
        fig.add_trace(go.Histogram(x=df[numeric_cols[3]], name=numeric_cols[3],
                                 marker_color='#d62728', nbinsx=50),
                    row=2, col=1)
    
    # 5. Correlation Heatmap
    if len(numeric_cols) > 0:
        corr_matrix = df[numeric_cols].corr()
        fig.add_trace(go.Heatmap(z=corr_matrix.values,
                               x=corr_matrix.columns,
                               y=corr_matrix.index,
                               colorscale='RdBu', zmid=0),
                    row=2, col=2)
    
    # 6. Missing Values Matrix
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        missing_cols = missing.index.tolist()
        missing_matrix = df[missing_cols].isnull().astype(int)
        fig.add_trace(go.Heatmap(z=missing_matrix.values.T,
                               x=missing_matrix.index,
                               y=missing_matrix.columns,
                               colorscale=['white', 'red']),
                    row=2, col=3)
    
    # 7. Feature Skewness
    if len(numeric_cols) > 0:
        numeric_features = df[numeric_cols]
        skewness = numeric_features.apply(lambda x: skew(x.dropna()))
        skewness = skewness.sort_values(ascending=False)
        top_skew = skewness.head(10)
        fig.add_trace(go.Bar(x=top_skew.index, y=top_skew.values,
                           marker_color='#9467bd', name='Skewness'),
                    row=3, col=1)
    
    # 8. Box Plot - Categorical Feature
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    if len(categorical_cols) > 0 and target_col:
        # Take first categorical column
        cat_col = categorical_cols[0]
        # Get unique values (limit to top 10)
        unique_vals = df[cat_col].value_counts().head(10).index.tolist()
        filtered_df = df[df[cat_col].isin(unique_vals)]
        
        box_data = []
        for val in unique_vals:
            box_data.append(filtered_df[filtered_df[cat_col] == val][target_col])
        
        fig.add_trace(go.Box(x=unique_vals, y=box_data,
                           name=cat_col, marker_color='#8c564b'),
                    row=3, col=2)
    
    # 9. Pair Plot - Top Features
    if len(numeric_cols) > 5 and target_col:
        # Get top correlated features with target
        if target_col in numeric_cols:
            corr_with_target = df[numeric_cols].corr()[target_col].abs().sort_values(ascending=False)
            top_features = corr_with_target.index[1:4].tolist()  # Top 3 excluding target itself
            top_features.append(target_col)
            
            # Create scatter matrix for top features
            pair_data = df[top_features].dropna()
            # We'll just show one scatter plot for simplicity
            if len(top_features) >= 2:
                fig.add_trace(go.Scatter(x=pair_data[top_features[0]], y=pair_data[top_features[1]],
                                      mode='markers',
                                      marker=dict(size=3, opacity=0.5,
                                                 color=pair_data[target_col] if target_col in pair_data.columns else 'blue'),
                                      showlegend=False),
                          row=3, col=3)
    
    # Update layout
    fig.update_layout(height=1200, showlegend=False,
                    title_text="📊 Comprehensive EDA Dashboard",
                    title_font_size=20)
    fig.update_xaxes(tickangle=45)
    
    fig.show()
    
    # Additional Visualizations
    print("\n ADDITIONAL ANALYSIS VISUALIZATIONS:")
    
    # Time-based analysis if available
    time_cols = [col for col in df.columns if 'Yr' in col or 'Year' in col]
    if len(time_cols) > 0 and target_col:
        fig2 = px.scatter(df, x=time_cols[0], y=target_col,
                        trendline="ols", title=f"Target over {time_cols[0]}")
        fig2.show()

# Run visualizations
create_visualizations(train, target)

# %% [markdown]
# ## 6.  **Advanced Feature Engineering**

# %%
# =========================
# Feature Engineering Pipeline
# =========================
class FeatureEngineer:
    """
    Advanced feature engineering class
    """
    def __init__(self, df):
        self.df = df.copy()
        self.numeric_features = []
        self.categorical_features = []
        
    def identify_features(self):
        """Identify feature types"""
        self.numeric_features = self.df.select_dtypes(include=[np.number]).columns.tolist()
        self.categorical_features = self.df.select_dtypes(include=['object']).columns.tolist()
        return self
    
    def create_basic_features(self):
        """Create basic engineered features"""
        print(" Creating basic features...")
        
        # Area-based features
        area_features = []
        if 'TotalBsmtSF' in self.df.columns:
            area_features.append('TotalBsmtSF')
        if '1stFlrSF' in self.df.columns:
            area_features.append('1stFlrSF')
        if '2ndFlrSF' in self.df.columns:
            area_features.append('2ndFlrSF')
        
        if len(area_features) == 3:
            self.df['TotalSF'] = self.df['TotalBsmtSF'] + self.df['1stFlrSF'] + self.df['2ndFlrSF']
        
        if 'LotArea' in self.df.columns and 'TotalSF' in self.df.columns:
            self.df['TotalArea'] = self.df['LotArea'] + self.df['TotalSF']
        
        # Bathroom features
        bath_features = []
        if 'FullBath' in self.df.columns:
            bath_features.append('FullBath')
        if 'HalfBath' in self.df.columns:
            bath_features.append('HalfBath')
        if 'BsmtFullBath' in self.df.columns:
            bath_features.append('BsmtFullBath')
        if 'BsmtHalfBath' in self.df.columns:
            bath_features.append('BsmtHalfBath')
        
        if len(bath_features) >= 2:
            self.df['TotalBath'] = self.df.get('FullBath', 0) + 0.5 * self.df.get('HalfBath', 0)
            if 'BsmtFullBath' in self.df.columns:
                self.df['TotalBath'] += self.df['BsmtFullBath']
            if 'BsmtHalfBath' in self.df.columns:
                self.df['TotalBath'] += 0.5 * self.df['BsmtHalfBath']
        
        # Age features
        if all(col in self.df.columns for col in ['YrSold', 'YearBuilt']):
            self.df['HouseAge'] = self.df['YrSold'] - self.df['YearBuilt']
        
        if all(col in self.df.columns for col in ['YrSold', 'YearRemodAdd']):
            self.df['RemodAge'] = self.df['YrSold'] - self.df['YearRemodAdd']
            self.df['IsRemodelled'] = (self.df['YearRemodAdd'] != self.df['YearBuilt']).astype(int)
        
        # Quality scores
        qual_cols = [col for col in self.df.columns if 'Qual' in col and col in self.numeric_features]
        if qual_cols:
            self.df['OverallQualScore'] = self.df[qual_cols].mean(axis=1)
        
        # Room features
        if all(col in self.df.columns for col in ['TotRmsAbvGrd', 'GrLivArea']):
            self.df['RoomDensity'] = self.df['TotRmsAbvGrd'] / self.df['GrLivArea'].clip(1)
        
        # Exterior features
        porch_cols = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
        existing_porch_cols = [col for col in porch_cols if col in self.df.columns]
        if existing_porch_cols:
            self.df['TotalPorch'] = self.df[existing_porch_cols].sum(axis=1)
        
        # Binary features
        binary_mappings = {
            'HasPool': 'PoolArea',
            'HasFireplace': 'Fireplaces',
            'HasGarage': 'GarageArea',
            'HasBasement': 'TotalBsmtSF',
            'Has2ndFloor': '2ndFlrSF'
        }
        
        for new_col, orig_col in binary_mappings.items():
            if orig_col in self.df.columns:
                self.df[new_col] = (self.df[orig_col] > 0).astype(int)
        
        return self
    
    def create_interaction_features(self):
        """Create interaction features"""
        print("🔧 Creating interaction features...")
        
        interactions = [
            ('OverallQual', 'GrLivArea'),
            ('TotalBath', 'GrLivArea'),
            ('OverallQual', 'TotalBath'),
            ('KitchenQual', 'GrLivArea'),
            ('GarageCars', 'GarageArea')
        ]
        
        for feat1, feat2 in interactions:
            if feat1 in self.df.columns and feat2 in self.df.columns:
                # Check if columns are numeric
                if feat1 in self.numeric_features and feat2 in self.numeric_features:
                    self.df[f'{feat1}_x_{feat2}'] = self.df[feat1] * self.df[feat2]
                    # Avoid division by zero
                    if (self.df[feat2] != 0).all():
                        self.df[f'{feat1}_div_{feat2}'] = self.df[feat1] / self.df[feat2].clip(1)
        
        return self
    
    def handle_skewness(self):
        """Handle skewed features"""
        print("🔧 Handling skewness...")
        
        for col in self.numeric_features:
            if col in self.df.columns:
                # Skip binary columns
                unique_vals = self.df[col].nunique()
                if unique_vals > 2:
                    # Handle NaN values
                    col_data = self.df[col].dropna()
                    if len(col_data) > 0:
                        try:
                            skewness = skew(col_data)
                            if abs(skewness) > 0.75:
                                # Apply log transformation (add 1 to handle zeros)
                                min_val = self.df[col].min()
                                if min_val <= 0:
                                    shift = abs(min_val) + 1
                                else:
                                    shift = 0
                                self.df[f'{col}_log'] = np.log1p(self.df[col] + shift)
                        except:
                            pass
        
        return self
    
    def get_engineered_data(self):
        """Return engineered dataframe"""
        print(f" Feature engineering complete! New shape: {self.df.shape}")
        return self.df

# Apply feature engineering
print("\n" + "="*60)
print(" ADVANCED FEATURE ENGINEERING")
print("="*60)

fe = FeatureEngineer(train_test)
fe.identify_features()
fe.create_basic_features()
fe.create_interaction_features()
fe.handle_skewness()
train_test_engineered = fe.get_engineered_data()

# Display new features
new_features = [col for col in train_test_engineered.columns if col not in train_test.columns]
print(f"\n Created {len(new_features)} new features:")
if new_features:
    print(new_features[:20])  # Show first 20
else:
    print("No new features created.")

# %% [markdown]
# ## 7.  **Advanced Data Preprocessing**

# %%
# =========================
# Data Preprocessing Pipeline
# =========================
class DataPreprocessor:
    """
    Advanced data preprocessing pipeline
    """
    def __init__(self, df):
        self.df = df.copy()
        self.numeric_cols = []
        self.categorical_cols = []
        self.scaler = None
        self.imputer_numeric = None
        self.imputer_categorical = None
        self.encoders = {}
        self.one_hot_columns = []
        
    def preprocess(self, target_col=None):
        """Main preprocessing pipeline"""
        print(" Starting advanced preprocessing...")
        
        # Store ID if exists
        if 'Id' in self.df.columns:
            self.ids = self.df['Id'].copy()
            self.df = self.df.drop('Id', axis=1)
        
        # Identify feature types
        self._identify_features()
        
        # Handle missing values
        self._handle_missing_values()
        
        # Encode categorical features
        self._encode_categorical()
        
        # Scale numeric features
        self._scale_numeric()
        
        # Remove outliers (optional)
        if target_col and target_col in self.df.columns:
            self._remove_outliers(target_col)
        
        print(f" Preprocessing complete! Final shape: {self.df.shape}")
        return self.df
    
    def _identify_features(self):
        """Identify numeric and categorical features"""
        self.numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        self.categorical_cols = self.df.select_dtypes(include=['object']).columns.tolist()
        print(f"Identified {len(self.numeric_cols)} numeric and {len(self.categorical_cols)} categorical features")
    
    def _handle_missing_values(self):
        """Handle missing values intelligently"""
        print(" Handling missing values...")
        
        # For numeric: use median
        if len(self.numeric_cols) > 0:
            self.imputer_numeric = SimpleImputer(strategy='median')
            # Get only numeric columns that exist in dataframe
            existing_numeric = [col for col in self.numeric_cols if col in self.df.columns]
            if existing_numeric:
                self.df[existing_numeric] = self.imputer_numeric.fit_transform(self.df[existing_numeric])
        
        # For categorical: use mode
        if len(self.categorical_cols) > 0:
            self.imputer_categorical = SimpleImputer(strategy='most_frequent')
            # Get only categorical columns that exist in dataframe
            existing_categorical = [col for col in self.categorical_cols if col in self.df.columns]
            if existing_categorical:
                self.df[existing_categorical] = self.imputer_categorical.fit_transform(self.df[existing_categorical])
    
    def _encode_categorical(self):
        """Encode categorical variables"""
        if len(self.categorical_cols) > 0:
            print(" Encoding categorical features...")
            
            # Get only categorical columns that exist in dataframe
            existing_categorical = [col for col in self.categorical_cols if col in self.df.columns]
            
            for col in existing_categorical:
                # Fill NaN with 'Missing' before encoding
                self.df[col] = self.df[col].fillna('Missing')
                
                if self.df[col].nunique() > 10:  # High cardinality
                    # Use LabelEncoder
                    try:
                        le = LabelEncoder()
                        self.df[col] = le.fit_transform(self.df[col].astype(str))
                        self.encoders[col] = le
                    except:
                        # If encoding fails, use frequency encoding
                        freq = self.df[col].value_counts(normalize=True)
                        self.df[col] = self.df[col].map(freq)
                else:  # Low cardinality
                    # One-Hot Encoding
                    dummies = pd.get_dummies(self.df[col], prefix=col, drop_first=True)
                    self.one_hot_columns.extend(dummies.columns.tolist())
                    self.df = pd.concat([self.df.drop(col, axis=1), dummies], axis=1)
            
            # Update numeric columns after encoding
            self.numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
            self.categorical_cols = []  # All categorical are now encoded
    
    def _scale_numeric(self):
        """Scale numeric features"""
        if len(self.numeric_cols) > 0:
            print(" Scaling numeric features...")
            self.scaler = RobustScaler()  # Robust to outliers
            # Get only numeric columns that exist in dataframe
            existing_numeric = [col for col in self.numeric_cols if col in self.df.columns]
            if existing_numeric:
                self.df[existing_numeric] = self.scaler.fit_transform(self.df[existing_numeric])
    
    def _remove_outliers(self, target_col):
        """Remove outliers using IQR method"""
        print(" Removing outliers...")
        if target_col in self.df.columns:
            Q1 = self.df[target_col].quantile(0.25)
            Q3 = self.df[target_col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            before = len(self.df)
            self.df = self.df[(self.df[target_col] >= lower_bound) & (self.df[target_col] <= upper_bound)]
            after = len(self.df)
            print(f"Removed {before - after} outliers ({((before-after)/before)*100:.1f}%)")
    
    def transform_new_data(self, new_df):
        """Transform new data using fitted preprocessors"""
        df_copy = new_df.copy()
        
        if 'Id' in df_copy.columns:
            new_ids = df_copy['Id'].copy()
            df_copy = df_copy.drop('Id', axis=1)
        
        # Handle missing values for numeric
        if self.imputer_numeric and len(self.numeric_cols) > 0:
            # Get numeric columns present in new data
            num_cols_present = [col for col in self.numeric_cols if col in df_copy.columns]
            if num_cols_present:
                df_copy[num_cols_present] = self.imputer_numeric.transform(df_copy[num_cols_present])
        
        # Handle missing values for categorical
        if self.imputer_categorical and len(self.categorical_cols) > 0:
            # Get categorical columns present in new data
            cat_cols_present = [col for col in self.categorical_cols if col in df_copy.columns]
            if cat_cols_present:
                df_copy[cat_cols_present] = self.imputer_categorical.transform(df_copy[cat_cols_present])
        
        # Encode categorical
        for col, encoder in self.encoders.items():
            if col in df_copy.columns:
                # Fill NaN and encode
                df_copy[col] = df_copy[col].fillna('Missing')
                try:
                    df_copy[col] = encoder.transform(df_copy[col].astype(str))
                except:
                    # Handle unseen labels
                    df_copy[col] = -1  # Assign special value
        
        # Create one-hot columns for new data
        for col in self.categorical_cols:
            if col in df_copy.columns:
                # Create one-hot encoding
                dummies = pd.get_dummies(df_copy[col], prefix=col)
                # Align with training columns
                for dummy_col in self.one_hot_columns:
                    if dummy_col in dummies.columns:
                        df_copy[dummy_col] = dummies[dummy_col]
                    else:
                        df_copy[dummy_col] = 0
                df_copy = df_copy.drop(col, axis=1)
        
        # Scale numeric
        if self.scaler and len(self.numeric_cols) > 0:
            num_cols_present = [col for col in self.numeric_cols if col in df_copy.columns]
            if num_cols_present:
                df_copy[num_cols_present] = self.scaler.transform(df_copy[num_cols_present])
        
        # Ensure all columns from training are present
        missing_cols = set(self.numeric_cols + self.one_hot_columns) - set(df_copy.columns)
        for col in missing_cols:
            df_copy[col] = 0
        
        # Reorder columns to match training
        expected_cols = self.numeric_cols + self.one_hot_columns
        df_copy = df_copy.reindex(columns=expected_cols, fill_value=0)
        
        return df_copy

# Apply preprocessing
print("\n" + "="*60)
print(" ADVANCED DATA PREPROCESSING")
print("="*60)

# Separate train and test
train_processed = train_test_engineered.iloc[:len(train)].copy()
test_processed = train_test_engineered.iloc[len(train):].copy()

# Add target back to train
if target in train.columns:
    train_processed[target] = y_train.values

# Initialize and fit preprocessor on training data (without target)
if target in train_processed.columns:
    X_train = train_processed.drop(target, axis=1)
else:
    X_train = train_processed.copy()

preprocessor = DataPreprocessor(X_train)
X_train_processed = preprocessor.preprocess()

# Get y_train (apply log transformation to target)
if target in train.columns:
    y_train_processed = np.log1p(train_processed[target])  # Log transform target
else:
    # Create sample target if not exists
    y_train_processed = np.log1p(np.random.lognormal(12, 0.3, len(X_train_processed)).clip(30000, 800000))

# Transform test data
X_test_processed = preprocessor.transform_new_data(test_processed)

print(f"\n Preprocessing Results:")
print(f"X_train shape: {X_train_processed.shape}")
print(f"X_test shape: {X_test_processed.shape}")
print(f"y_train shape: {y_train_processed.shape}")

# %% [markdown]
# ## 8.  **Advanced Model Training & Evaluation**

# %%
# =========================
# Model Training Pipeline
# =========================
class ModelTrainer:
    """
    Advanced model training and evaluation
    """
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.models = {}
        self.results = {}
        self.best_model = None
        self.best_score = np.inf
        
    def train_multiple_models(self):
        """Train multiple ML models"""
        print(" Training multiple models...")
        
        # Define models with initial parameters
        self.models = {
            'Linear Regression': LinearRegression(),
            'Ridge Regression': Ridge(alpha=1.0, random_state=42),
            'Lasso Regression': Lasso(alpha=0.001, random_state=42, max_iter=10000),
            'ElasticNet': ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=42, max_iter=10000),
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'XGBoost': XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, verbosity=0, n_jobs=-1),
            'LightGBM': LGBMRegressor(n_estimators=100, random_state=42, verbose=-1, n_jobs=-1),
            'SVR': SVR(kernel='rbf')
        }
        
        # 5-Fold Cross Validation
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        
        for name, model in self.models.items():
            print(f"  Training {name}...")
            
            try:
                # Cross validation scores
                cv_scores = cross_val_score(model, self.X_train, self.y_train,
                                          cv=kf, scoring='neg_mean_squared_error',
                                          n_jobs=-1)
                
                # Calculate RMSE
                rmse_scores = np.sqrt(-cv_scores)
                
                # Store results
                self.results[name] = {
                    'model': model,
                    'cv_rmse_mean': rmse_scores.mean(),
                    'cv_rmse_std': rmse_scores.std(),
                    'cv_scores': rmse_scores
                }
                
                # Fit model on full training data
                model.fit(self.X_train, self.y_train)
                
                # Update best model
                if rmse_scores.mean() < self.best_score:
                    self.best_score = rmse_scores.mean()
                    self.best_model = name
                    
            except Exception as e:
                print(f" Error training {name}: {str(e)[:100]}...")
                # Skip this model
                continue
        
        return self
    
    def hyperparameter_tuning(self, model_name):
        """Perform hyperparameter tuning for a specific model"""
        print(f" Tuning hyperparameters for {model_name}...")
        
        if model_name not in self.models:
            print(f"Model {model_name} not found.")
            return None
        
        if model_name == 'Random Forest':
            param_grid = {
                'n_estimators': [100, 200],
                'max_depth': [10, 20, None],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2],
                'max_features': ['auto', 'sqrt']
            }
            model = RandomForestRegressor(random_state=42, n_jobs=-1)
            
        elif model_name == 'XGBoost':
            param_grid = {
                'n_estimators': [100, 200],
                'learning_rate': [0.01, 0.1],
                'max_depth': [3, 5],
                'subsample': [0.8, 1.0],
                'colsample_bytree': [0.8, 1.0]
            }
            model = XGBRegressor(random_state=42, verbosity=0, n_jobs=-1)
            
        elif model_name == 'LightGBM':
            param_grid = {
                'n_estimators': [100, 200],
                'learning_rate': [0.01, 0.1],
                'num_leaves': [31, 50],
                'max_depth': [5, 10],
                'subsample': [0.8, 1.0]
            }
            model = LGBMRegressor(random_state=42, verbose=-1, n_jobs=-1)
        
        else:
            print(f"No tuning grid for {model_name}")
            return None
        
        # Randomized Search
        try:
            random_search = RandomizedSearchCV(
                model, param_grid, n_iter=10,
                cv=3, scoring='neg_mean_squared_error',
                random_state=42, n_jobs=-1, verbose=0
            )
            
            random_search.fit(self.X_train, self.y_train)
            
            # Update model with best parameters
            self.models[model_name] = random_search.best_estimator_
            if model_name in self.results:
                self.results[model_name]['best_params'] = random_search.best_params_
                self.results[model_name]['best_score'] = np.sqrt(-random_search.best_score_)
            
            print(f"Best parameters: {random_search.best_params_}")
            print(f"Best RMSE: {np.sqrt(-random_search.best_score_):.4f}")
            
            return random_search.best_estimator_
            
        except Exception as e:
            print(f" Error tuning {model_name}: {str(e)[:100]}...")
            return None
    
    def create_ensemble(self):
        """Create ensemble model"""
        print(" Creating ensemble model...")
        
        # Get top models that were successfully trained
        if len(self.results) == 0:
            print("No models to ensemble.")
            return None, None
        
        sorted_results = sorted(self.results.items(), key=lambda x: x[1]['cv_rmse_mean'])
        top_models = sorted_results[:min(3, len(sorted_results))]
        
        if len(top_models) == 0:
            return None, None
        
        # Create voting regressor
        estimators = [(name, self.models[name]) for name, _ in top_models]
        
        try:
            voting_reg = VotingRegressor(estimators=estimators)
            
            # Cross validate ensemble
            kf = KFold(n_splits=3, shuffle=True, random_state=42)
            cv_scores = cross_val_score(voting_reg, self.X_train, self.y_train,
                                      cv=kf, scoring='neg_mean_squared_error')
            rmse_scores = np.sqrt(-cv_scores)
            
            # Store ensemble
            voting_reg.fit(self.X_train, self.y_train)
            self.models['Ensemble (Voting)'] = voting_reg
            self.results['Ensemble (Voting)'] = {
                'model': voting_reg,
                'cv_rmse_mean': rmse_scores.mean(),
                'cv_rmse_std': rmse_scores.std(),
                'cv_scores': rmse_scores
            }
            
            # Create stacking regressor
            stacking_reg = StackingRegressor(
                estimators=estimators,
                final_estimator=LinearRegression(),
                cv=3
            )
            
            stacking_reg.fit(self.X_train, self.y_train)
            self.models['Ensemble (Stacking)'] = stacking_reg
            
            return voting_reg, stacking_reg
            
        except Exception as e:
            print(f" Error creating ensemble: {str(e)[:100]}...")
            return None, None
    
    def evaluate_models(self):
        """Evaluate and compare all models"""
        print("\n" + "="*80)
        print(" MODEL COMPARISON RESULTS")
        print("="*80)
        
        if len(self.results) == 0:
            print("No models to evaluate.")
            return pd.DataFrame()
        
        # Create results dataframe
        results_data = []
        for name, res in self.results.items():
            results_data.append({
                'Model': name,
                'CV RMSE Mean': res['cv_rmse_mean'],
                'CV RMSE Std': res['cv_rmse_std'],
                'CV RMSE Min': res['cv_scores'].min(),
                'CV RMSE Max': res['cv_scores'].max()
            })
        
        results_df = pd.DataFrame(results_data).sort_values('CV RMSE Mean')
        
        # Display with styling
        try:
            display(results_df.style.background_gradient(subset=['CV RMSE Mean'], cmap='RdYlGn_r'))
        except:
            display(results_df)
        
        # Visualize results
        fig = go.Figure()
        
        models = results_df['Model']
        means = results_df['CV RMSE Mean']
        stds = results_df['CV RMSE Std']
        
        fig.add_trace(go.Bar(
            x=models, y=means,
            error_y=dict(type='data', array=stds),
            marker_color='lightblue',
            name='CV RMSE'
        ))
        
        fig.update_layout(
            title='Model Comparison - Cross Validation RMSE',
            xaxis_title='Model',
            yaxis_title='RMSE',
            template='plotly_white',
            height=500
        )
        
        fig.show()
        
        if self.best_model:
            print(f"\n Best Model: {self.best_model} (RMSE: {self.best_score:.4f})")
        
        return results_df
    
    def get_best_model(self):
        """Get the best performing model"""
        if self.best_model and self.best_model in self.models:
            return self.models[self.best_model], self.best_model
        elif len(self.models) > 0:
            # Return first available model
            first_model_name = list(self.models.keys())[0]
            return self.models[first_model_name], first_model_name
        else:
            return None, "No model"

# Train models
print("\n" + "="*60)
print(" ADVANCED MODEL TRAINING")
print("="*60)

trainer = ModelTrainer(X_train_processed, y_train_processed)
trainer.train_multiple_models()

# Hyperparameter tuning for top models
if 'Random Forest' in trainer.models:
    trainer.hyperparameter_tuning('Random Forest')
if 'XGBoost' in trainer.models:
    trainer.hyperparameter_tuning('XGBoost')
if 'LightGBM' in trainer.models:
    trainer.hyperparameter_tuning('LightGBM')

# Create ensemble models
trainer.create_ensemble()

# Evaluate all models
results_df = trainer.evaluate_models()

# Get best model
best_model, best_model_name = trainer.get_best_model()

# %% [markdown]
# ## 9.  **Model Interpretation & Feature Importance**

# %%
# =========================
# Feature Importance Analysis
# =========================
def perform_feature_analysis(model, X, model_name="Model"):
    """
    Perform feature importance analysis
    """
    print(f"\n Performing feature analysis for {model_name}...")
    
    try:
        # Check if model has feature_importances_ attribute
        if hasattr(model, 'feature_importances_'):
            # Get feature importance
            feature_importance = pd.DataFrame({
                'feature': X.columns,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            print(" Feature Importance Plot:")
            
            # Plot top 20 features
            plt.figure(figsize=(12, 8))
            top_features = feature_importance.head(20)
            sns.barplot(x='importance', y='feature', data=top_features)
            plt.title(f'Feature Importance - {model_name}', fontsize=16)
            plt.xlabel('Importance', fontsize=12)
            plt.ylabel('Feature', fontsize=12)
            plt.tight_layout()
            plt.show()
            
            print("\n Top 10 Most Important Features:")
            display(feature_importance.head(10))
            
            return feature_importance
            
        elif hasattr(model, 'coef_'):
            # For linear models
            coefficients = pd.DataFrame({
                'feature': X.columns,
                'coefficient': model.coef_
            }).sort_values('coefficient', key=abs, ascending=False)
            
            print(" Coefficient Plot:")
            
            # Plot top 20 coefficients
            plt.figure(figsize=(12, 8))
            top_coeff = coefficients.head(20)
            colors = ['red' if x < 0 else 'green' for x in top_coeff['coefficient']]
            plt.barh(top_coeff['feature'], top_coeff['coefficient'], color=colors)
            plt.title(f'Feature Coefficients - {model_name}', fontsize=16)
            plt.xlabel('Coefficient Value', fontsize=12)
            plt.ylabel('Feature', fontsize=12)
            plt.tight_layout()
            plt.show()
            
            print("\n Top 10 Most Influential Features:")
            display(coefficients.head(10))
            
            return coefficients
            
        else:
            print(" Model doesn't have feature importance or coefficients attribute.")
            
            # Try permutation importance as fallback
            from sklearn.inspection import permutation_importance
            
            try:
                perm_importance = permutation_importance(model, X, y_train_processed, 
                                                       n_repeats=10, random_state=42)
                
                perm_df = pd.DataFrame({
                    'feature': X.columns,
                    'importance': perm_importance.importances_mean
                }).sort_values('importance', ascending=False)
                
                print(" Permutation Importance Plot:")
                
                plt.figure(figsize=(12, 8))
                top_perm = perm_df.head(20)
                sns.barplot(x='importance', y='feature', data=top_perm)
                plt.title(f'Permutation Importance - {model_name}', fontsize=16)
                plt.xlabel('Importance', fontsize=12)
                plt.ylabel('Feature', fontsize=12)
                plt.tight_layout()
                plt.show()
                
                print("\n Top 10 Most Important Features (Permutation):")
                display(perm_df.head(10))
                
                return perm_df
                
            except:
                print(" Could not compute feature importance for this model.")
                return None
                
    except Exception as e:
        print(f" Feature analysis failed: {str(e)[:100]}...")
        return None

# Perform feature analysis on best model
if best_model:
    feature_importance = perform_feature_analysis(best_model, X_train_processed, best_model_name)
else:
    print(" No model available for feature analysis.")

# %% [markdown]
# ## 10.  **Final Predictions & Submission**

# %%
# =========================
# Final Predictions
# =========================
print("\n" + "="*60)
print(" MAKING FINAL PREDICTIONS")
print("="*60)

if best_model:
    # Make predictions
    y_pred_train = best_model.predict(X_train_processed)
    y_pred_test = best_model.predict(X_test_processed)
    
    # Inverse log transformation
    y_pred_train_exp = np.expm1(y_pred_train)
    y_pred_test_exp = np.expm1(y_pred_test)
    y_train_exp = np.expm1(y_train_processed)
    
    # Calculate final metrics
    train_rmse = np.sqrt(mean_squared_error(y_train_exp, y_pred_train_exp))
    train_mae = mean_absolute_error(y_train_exp, y_pred_train_exp)
    train_r2 = r2_score(y_train_exp, y_pred_train_exp)
    
    print("\n FINAL MODEL PERFORMANCE:")
    print("-" * 40)
    print(f"Best Model: {best_model_name}")
    print(f"Train RMSE: ${train_rmse:,.2f}")
    print(f"Train MAE: ${train_mae:,.2f}")
    print(f"Train R² Score: {train_r2:.4f}")
    
    # Calculate MAPE if no zero values
    if (y_train_exp != 0).all():
        mape = np.mean(np.abs((y_train_exp - y_pred_train_exp) / y_train_exp)) * 100
        print(f"Train MAPE: {mape:.2f}%")
    
    # Residual Analysis
    residuals = y_train_exp - y_pred_train_exp
    print(f"\n RESIDUAL ANALYSIS:")
    print(f"Residual Mean: ${residuals.mean():,.2f}")
    print(f"Residual Std: ${residuals.std():,.2f}")
    
    # Plot residuals
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Residuals vs Predicted
    axes[0, 0].scatter(y_pred_train_exp, residuals, alpha=0.5)
    axes[0, 0].axhline(y=0, color='r', linestyle='--')
    axes[0, 0].set_xlabel('Predicted Values')
    axes[0, 0].set_ylabel('Residuals')
    axes[0, 0].set_title('Residuals vs Predicted')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Distribution of residuals
    axes[0, 1].hist(residuals, bins=50, edgecolor='black')
    axes[0, 1].set_xlabel('Residuals')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Distribution of Residuals')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Q-Q plot
    try:
        stats.probplot(residuals, dist="norm", plot=axes[1, 0])
        axes[1, 0].set_title('Q-Q Plot of Residuals')
        axes[1, 0].grid(True, alpha=0.3)
    except:
        axes[1, 0].text(0.5, 0.5, 'Q-Q Plot not available', 
                       ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Q-Q Plot of Residuals')
    
    # Actual vs Predicted
    axes[1, 1].scatter(y_train_exp, y_pred_train_exp, alpha=0.5)
    axes[1, 1].plot([y_train_exp.min(), y_train_exp.max()], 
                   [y_train_exp.min(), y_train_exp.max()], 'r--', lw=2)
    axes[1, 1].set_xlabel('Actual Values')
    axes[1, 1].set_ylabel('Predicted Values')
    axes[1, 1].set_title('Actual vs Predicted')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Create submission file
    if test_ids is not None:
        submission = pd.DataFrame({
            'Id': test_ids,
            'SalePrice': y_pred_test_exp
        })
        
        # Save submission
        submission_file = 'submission_final.csv'
        submission.to_csv(submission_file, index=False)
        print(f"\n Submission file saved: {submission_file}")
        
        # Display sample predictions
        print("\n SAMPLE PREDICTIONS:")
        print("-" * 40)
        sample_predictions = submission.head(10).copy()
        sample_predictions['SalePrice'] = sample_predictions['SalePrice'].apply(lambda x: f"${x:,.0f}")
        display(sample_predictions)
    else:
        print(" No test IDs available for submission.")
        
else:
    print(" No model available for predictions.")

# %% [markdown]
# ## 11.  **Model Saving & Deployment**

# %%
# =========================
# Save Model & Artifacts
# =========================
print("\n" + "="*60)
print(" SAVING MODEL & ARTIFACTS")
print("="*60)

# Create artifacts directory
os.makedirs('model_artifacts', exist_ok=True)

if best_model:
    # Save best model
    model_file = 'model_artifacts/best_model.pkl'
    try:
        joblib.dump(best_model, model_file)
        print(f" Best model saved: {model_file}")
    except Exception as e:
        print(f" Error saving model: {str(e)[:100]}...")

# Save preprocessor
if 'preprocessor' in locals():
    preprocessor_file = 'model_artifacts/preprocessor.pkl'
    try:
        joblib.dump(preprocessor, preprocessor_file)
        print(f" Preprocessor saved: {preprocessor_file}")
    except Exception as e:
        print(f" Error saving preprocessor: {str(e)[:100]}...")

# Save feature importance
if 'feature_importance' in locals() and feature_importance is not None:
    importance_file = 'model_artifacts/feature_importance.csv'
    try:
        feature_importance.to_csv(importance_file, index=False)
        print(f" Feature importance saved: {importance_file}")
    except Exception as e:
        print(f" Error saving feature importance: {str(e)[:100]}...")

# Save model metadata
metadata = {
    'model_name': best_model_name if best_model else 'No model',
    'train_rmse': float(train_rmse) if best_model else 0.0,
    'train_r2': float(train_r2) if best_model else 0.0,
    'train_mae': float(train_mae) if best_model else 0.0,
    'features_used': X_train_processed.shape[1] if 'X_train_processed' in locals() else 0,
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_parameters': str(best_model.get_params()) if best_model and hasattr(best_model, 'get_params') else 'N/A'
}

metadata_file = 'model_artifacts/model_metadata.json'
try:
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=4)
    print(f" Model metadata saved: {metadata_file}")
except Exception as e:
    print(f" Error saving metadata: {str(e)[:100]}...")

# Create a simple deployment script
deployment_script = """
# =========================
# House Price Prediction API
# =========================
import joblib
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify

app = Flask(__name__)

# Load model and preprocessor
try:
    model = joblib.load('model_artifacts/best_model.pkl')
    preprocessor = joblib.load('model_artifacts/preprocessor.pkl')
    print(" Model and preprocessor loaded successfully")
except:
    print(" Could not load model artifacts")
    model = None
    preprocessor = None

@app.route('/predict', methods=['POST'])
def predict():
    if model is None or preprocessor is None:
        return jsonify({
            'status': 'error',
            'message': 'Model not loaded properly'
        }), 500
    
    try:
        # Get data from request
        data = request.get_json()
        
        # Convert to DataFrame
        input_data = pd.DataFrame([data])
        
        # Preprocess
        processed_data = preprocessor.transform_new_data(input_data)
        
        # Predict
        prediction_log = model.predict(processed_data)
        
        # Convert to actual price
        prediction = np.expm1(prediction_log[0])
        
        return jsonify({
            'predicted_price': float(prediction),
            'status': 'success',
            'message': 'Prediction successful'
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        }), 400

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy'})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
"""

# Save deployment script
try:
    with open('model_artifacts/deployment_api.py', 'w') as f:
        f.write(deployment_script)
    print(" Deployment API script created: model_artifacts/deployment_api.py")
except Exception as e:
    print(f" Error saving deployment script: {str(e)[:100]}...")

# Create requirements file
requirements = """
pandas>=1.5.0
numpy>=1.24.0
scikit-learn>=1.2.0
xgboost>=1.7.0
lightgbm>=3.3.0
joblib>=1.2.0
flask>=2.0.0
"""

try:
    with open('model_artifacts/requirements.txt', 'w') as f:
        f.write(requirements)
    print(" Requirements file created: model_artifacts/requirements.txt")
except Exception as e:
    print(f" Error saving requirements: {str(e)[:100]}...")

print("\n All artifacts saved successfully!")
print(" Check the 'model_artifacts' folder for all saved files.")

# %% [markdown]
# ## 12.  **Project Summary & Next Steps**

# %%
# =========================
# Project Summary
# =========================
print("\n" + "="*80)
print(" PROJECT SUMMARY & ACHIEVEMENTS")
print("="*80)

# Prepare summary data
summary_data = []
summary_data.append(["1. Data Loading", " Completed", f"{train.shape} samples, {train.shape[1]-1 if target in train.columns else train.shape[1]} features"])
summary_data.append(["2. EDA", " Completed", f"{len(missing_values) if len(missing_values) > 0 else 0} features with missing values"])
summary_data.append(["3. Feature Engineering", " Completed", f"{len(new_features) if 'new_features' in locals() else 0} new features created"])
summary_data.append(["4. Preprocessing", " Completed", f"{X_train_processed.shape[1] if 'X_train_processed' in locals() else 0} final features"])
summary_data.append(["5. Model Training", " Completed", f"{len(trainer.models) if 'trainer' in locals() else 0} models trained"])
summary_data.append(["6. Model Evaluation", " Completed", f"Best RMSE: ${train_rmse:,.2f if best_model else 'N/A'}, R²: {train_r2:.4f if best_model else 'N/A'}"])
summary_data.append(["7. Feature Analysis", " Completed", "Feature importance analysis completed"])
summary_data.append(["8. Predictions", " Completed", f"Predictions for {len(test) if 'test' in locals() else 0} samples"])
summary_data.append(["9. Deployment", " Ready", "API & artifacts ready for deployment"])

summary_df = pd.DataFrame(summary_data, columns=["Project Phase", "Status", "Key Metrics"])
display(summary_df)

print("\n KEY ACHIEVEMENTS:")
print("-" * 40)
print("1.  Comprehensive EDA with interactive visualizations")
print("2.  Advanced feature engineering (created new features)")
print("3.  Multiple ML models with hyperparameter tuning")
print("4.  Ensemble methods (Voting & Stacking)")
print("5.  Feature importance analysis")
print("6.  Residual analysis and validation")
print("7.  Complete deployment package")
print("8.  Professional documentation and artifacts")

print("\n NEXT STEPS FOR PRODUCTION:")
print("-" * 40)
print("1. Deploy API using: python model_artifacts/deployment_api.py")
print("2. Containerize with Docker for cloud deployment")
print("3. Create Streamlit dashboard for business users")
print("4. Set up CI/CD pipeline with GitHub Actions")
print("5. Monitor model performance with MLflow")
print("6. Implement A/B testing for model updates")

print("\n FOR INTERNSHIP EXCELLENCE:")
print("-" * 40)
print("1.  Push code to GitHub with professional README")
print("2.  Write Medium article explaining your approach")
print("3.  Create interactive dashboard with Streamlit")
print("4.  Share on LinkedIn with #DataScience #MachineLearning")
print("5.  Prepare presentation for technical interview")

print("\n" + "="*80)
print(" CONGRATULATIONS! You've built a production-ready ML project!")
print("="*80)

# Display final message
print("\n" + " " * 40)
print(" Your project is now COMPLETE and READY FOR INTERNSHIP! ✨")
print("" * 40)
print("\nYou have successfully implemented:")
print("  • Advanced EDA & Feature Engineering")
print("  • Multiple ML Models with Hyperparameter Tuning")
print("  • Ensemble Learning Methods")
print("  • Feature Importance Analysis")
print("  • Complete Deployment Package")
print("\n Check the 'model_artifacts' folder for all saved files.")
print(" Run 'python model_artifacts/deployment_api.py' to start the API.")
print("\nGood luck with your internship application! ")

 All libraries imported successfully!
 Train data loaded from: ../data/train.csv
 Test data loaded from: ../data/test.csv

 Data Summary:
Train shape: (1460, 81)
Test shape: (1459, 80)
Combined shape: (2919, 80)

 First 5 rows of training data:


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


 COMPREHENSIVE EDA REPORT

 BASIC INFORMATION
----------------------------------------
Shape: (1460, 81)
Memory Usage: 0.90 MB

 DATA TYPES
----------------------------------------
object: 43
int64: 35
float64: 3

 MISSING VALUES ANALYSIS
----------------------------------------


,Missing_Count,Missing_Percent
PoolQC,1453,99.521
MiscFeature,1406,96.301
Alley,1369,93.767
Fence,1179,80.753
MasVnrType,872,59.726
FireplaceQu,690,47.260
LotFrontage,259,17.740
GarageType,81,5.548
GarageYrBlt,81,5.548
GarageFinish,81,5.548



 TARGET VARIABLE ANALYSIS
----------------------------------------
Target: SalePrice
Mean: $180,921.20
Median: $163,000.00
Std: $79,442.50
Min: $34,900.00
Max: $755,000.00
Skewness: 1.8829
Kurtosis: 6.5363

 STATISTICAL SUMMARY
----------------------------------------


...

 CREATING ADVANCED VISUALIZATIONS...



 ADDITIONAL ANALYSIS VISUALIZATIONS:



 ADVANCED FEATURE ENGINEERING
 Creating basic features...
🔧 Creating interaction features...
🔧 Handling skewness...
 Feature engineering complete! New shape: (2919, 119)

 Created 39 new features:
['TotalSF', 'TotalArea', 'TotalBath', 'HouseAge', 'RemodAge', 'IsRemodelled', 'OverallQualScore', 'RoomDensity', 'TotalPorch', 'HasPool', 'HasFireplace', 'HasGarage', 'HasBasement', 'Has2ndFloor', 'OverallQual_x_GrLivArea', 'OverallQual_div_GrLivArea', 'GarageCars_x_GarageArea', 'MSSubClass_log', 'LotFrontage_log', 'LotArea_log']

 ADVANCED DATA PREPROCESSING
 Starting advanced preprocessing...
Identified 75 numeric and 43 categorical features
 Handling missing values...
 Encoding categorical features...
 Scaling numeric features...
 Preprocessing complete! Final shape: (1460, 233)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Exterior1st
- Exterior2nd
- Neighborhood


: 

ImportError: Numba needs NumPy 2.3 or less. Got NumPy 2.4.